In [ ]:
import geopandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
file_salud = "data/denue_salud/conjunto_de_datos/denue_inegi_62_.shp"

In [ ]:
sector_salud = geopandas.read_file(file_salud)

In [ ]:
sector_salud.info()

In [ ]:
sector_salud.head()

In [ ]:
sector_salud.plot()

In [ ]:
sector_salud.nombre_act.value_counts()

In [ ]:
sector_salud.codigo_act = sector_salud.codigo_act.astype(np.int64)

In [ ]:
hospitales = sector_salud.loc[sector_salud.codigo_act.between(622000, 623000), ]

In [ ]:
hospitales.shape

In [ ]:
hospitales.plot()

In [ ]:
hospitales.nombre_act.value_counts().plot.bar()

In [ ]:
hospitales["sector"] = hospitales["codigo_act"].apply(lambda codigo: 'Sector público' if codigo % 2 == 0 
                                                   else 'Sector privado')

In [ ]:
hospitales_entidad_sector = hospitales.groupby(["entidad", "sector"]).count()["id"].reset_index(name='count')

In [ ]:
hospitales_entidad_sector = hospitales_entidad_sector.pivot(index='entidad', columns='sector', values='count')
hospitales_entidad_sector.head()

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]
hospitales_entidad_sector.plot.bar(stacked = True)

In [ ]:
hospitales_entidad_sector_percent = hospitales_entidad_sector.apply(lambda x: x*100/sum(x), axis=1)
hospitales_entidad_sector_percent.head()

In [ ]:
hospitales_entidad_sector_percent.plot.bar(stacked = True)

In [ ]:
file_marco = "data/marco_geoestadistico/CONTINUO_NACIONAL/ESTADOS.shp"
geo_estados = geopandas.read_file(file_marco)

In [ ]:
geo_estados.info()

In [ ]:
geo_estados.plot()